# CS 271 CCSR

In [1]:
# Define imports
import random
import pandas as pd
import numpy as np
# import tensorflow as tf
from numpy import asarray
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
# from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sys
import pickle
import os
import csv
from collections import defaultdict
import time
#
import re

## Step 1: Read in relevant Data Files

Make sure the filepaths below correspond to your local copies of the files

In [2]:
# Define constants
PATIENT_DATA_FILE = 'B220_SAA_v1.csv'
CLEANED_LABELS_FILE = 'ICD_Label_Cleaned_Oct_25.csv'
CODE_DESC_FILE = 'BIODS220_ICD_Dx_10_9_v7 - icd_dx_10_9_v7.csv'
ICD10_to_CCSR = 'ICD10-CCSR.csv'
LANG_MODEL = 'CCSR'

In [3]:
def read_csv_to_dict(file_path: str, key: int, value: int):
    ret_dict = {}
    with open(file_path, newline='') as csvfile:
        data = csv.reader(csvfile, delimiter=',')
        for row in data:
            ret_dict[row[key]] = row[value]
    print("Reading {} complete!".format(file_path))
    return ret_dict

In [4]:
def get_category_dict():
    category_dict = {
        'Circulatory': 0,
        'Dermatologic': 4,
        'Endocrine & Immune': 6,
        'Gastrointestinal': 1,
        'Genitourinary': 1, 
        'Hematologic': 4,
        'Infectious': 6,
        'Injury': 2,
        'Injury & Poisoning': 2,
        'Poisoning': 2,
        'Musculoskeletal': 2,
        'Neurologic': 3,
        'Other': 4,
        'Obstetric': 5,
        'Neoplastic': 4,
        'Psychiatric': 3,
        'Respiratory': 0,
        'Substance use': 2}
    #use to_categorical()
    return category_dict

In [5]:
NUM_CLASSES = 7

In [6]:
def get_label2className_dict():
    '''
    Function to create reverse map of labels to category name
    '''
    className_dict = {
        0:'Cardio-Resp',
        1:'Abdominal',
        2:'Injury/Subst/Poison', 
        3:'NeuroPsych',
        4:'Other',
        5:'Obstetric',
        6:'Infection-Immune'
    }
    #use to_categorical()
    return className_dict

In [7]:
# Create labels dict i.e. code -> label, i.e. A840 -> 'Neurologic'
label_dict = read_csv_to_dict(CLEANED_LABELS_FILE, key=0, value=1)

# Create descriptions dict i.e. code -> description
codes_dict = read_csv_to_dict(CODE_DESC_FILE, key=0, value=2)

# Create mapping from ICD10 code to CCSR category
ccsr_dict = read_csv_to_dict(ICD10_to_CCSR, key=0, value=1)

# Create dict for label to int
category_dict = get_category_dict()

Reading ICD_Label_Cleaned_Oct_25.csv complete!
Reading BIODS220_ICD_Dx_10_9_v7 - icd_dx_10_9_v7.csv complete!
Reading ICD10-CCSR.csv complete!


## Step 2: Create one-hot feature vectors

First, let's take a look at the table of ICD10 codes to CCSR categories.

In [8]:
df = pd.read_csv(ICD10_to_CCSR, usecols = ['ICD-10-CM Code', 'CCSR Category'])
df

,ICD-10-CM Code,CCSR Category
0,A000,DIG001
1,A000,INF003
2,A001,DIG001
3,A001,INF003
4,A009,DIG001
...,...,...
83452,Z9912,FAC012
83453,Z992,FAC025
83454,Z993,FAC025
83455,Z9981,FAC025


In [9]:
df.nunique()

ICD-10-CM Code    73205
CCSR Category       540
dtype: int64

We can see that we have over 83,000 ICD-10 codes that are assigned to one of 540 categories, significantly reducing the dimensionality of the data! We're next going to one hot encode each ICD-10 code into a vector of 540 entries, each corresponding to a particular CCSR category.

In [10]:
ccsr_one_hot = pd.get_dummies(df['CCSR Category']).to_numpy()
ccsr_one_hot.shape

(83457, 540)

ICD codes may map to more than one CCSR Category. In this case, we want to ensure that the one-hot encoding includes multiple CCSR categories. To accomplish this, we will create a dictionary that maps an ICD code to the corresponding 540-vector encoding. We should have 73,205 keys in the dictionary.

In [11]:
icd_to_ccsr = {} # Maps icd-10 code to ccsr category (which is 540-vector)

for i in range(len(ccsr_one_hot)):
    icd_code = df.iloc[i, 0] # 'A000'
    encoding = ccsr_one_hot[i,:] # one-hot encoding of size (540,)
    
    try:
        prev_encoding = icd_to_ccsr[icd_code]
        # if a previous encoding already exists, 
        # then we concatenate it with the one-hot encoding of the current category
        encoding = np.sum([prev_encoding, encoding], axis=0)
    except:
        pass
    
    icd_to_ccsr[icd_code] = encoding


Make sure we have one key per each unique ICD code. Thus, we are mapping every key to a (540,) one-hot encoding vector.

In [12]:
assert(len(icd_to_ccsr.keys()) == df['ICD-10-CM Code'].nunique())
ccsr_one_hot = None

For the sake of reducing feature space, we are only including three features in our embedding. Future iterations of our embeddings will include more features, such as the patient's county.

In [13]:
def create_one_hot(patient_data):
    # Creates one-hot vectors
    columns_to_one_hot = ['Sex','Race']
    one_hot = pd.get_dummies(patient_data[columns_to_one_hot])
    
    ordinal_columns = ['Age']
    one_hot = pd.concat([patient_data[ordinal_columns], one_hot], axis=1)
    
    # Normalize age
    x = one_hot.Age.values.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    one_hot.Age = x_scaled.astype(np.int16)
    
    return one_hot.values

## Step 3: Create one-hot ICD code encoding

Given an ICD code, we retrieve the one hot encoding (540x1).

In [14]:
def code_to_embedding(code):
    try:
        embedding = icd_to_ccsr[code]
    except:
        print(code)
        return np.zeros(540)
    return embedding

For each row of ICD Codes (i.e ['E839', 'SA920']), we retrieve the corresponding embeddings. For patient visits with n ICD Codes where n>1, we give 0.75 weight to the primary ICD code and 0.25/n-1 weight to the remaining (secondary) codes.

In [15]:
def invalidCode(code):
    numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    return code[0] in numbers

In [16]:
def rowToEncoding(row):
    """
    input_row: A list of ICD10 codes
    returns a 540x1 embedding
    """
    patient_ICD_codes = [entry for entry in row[16:41] if entry != '']
    n = len(patient_ICD_codes)
    code = patient_ICD_codes[0]
    if invalidCode(code):
        print("Invalid primary code: ", code)
        return
    # Primary ICD code:
    primary_embedding = code_to_embedding(code)
    if n < 2: return primary_embedding
    
    # Subsequent ICD codes:
    secondary_embedding = None
    for i in range(1, n):
        code = patient_ICD_codes[i]
        if invalidCode(code):
            print("Invalid secondary code: ", code)
            continue
        curr_embed = code_to_embedding(code)
        if secondary_embedding is None:
            secondary_embedding = curr_embed
        else:
            secondary_embedding = np.sum([secondary_embedding, curr_embed], axis=0)
    
    
    return np.sum([primary_embedding, secondary_embedding], axis=0)

**Important:** Since a patient may have more than one ICD-10 code attributed to their visit to the emergency room, two or more codes may share the same CCSR category. Thus, when we take the sum of the CCSR one-hot encodings, we may end up with numbers greater than 1. Since we ultimately want the 540-vector encoding of the patient's visit to be a binary representation of the CCSR categories, numbers greater than 1 should be reduced to 1. This case will only occur when a patient comes in with more than one ICD_10 code that shares the same CCSR category, in which case we will be taking the sum of two identical one-hot encodings, producing something similar to [0, ..., 2, ...0].

We read in the patient visit csv file again and create embeddings for each visit as we read the file. For each visit, we have already computed the one-hot vector encoding for categorical and ordinal variables. Here, we combine those encodings with ICD_10 code embeddings. To reduce model complexity, we limit the number of patients (not total visits) for which to create embeddings. Here, we are working under the assumption that all visits for a patient appear one after the other in the csv file, thus. Takes a few mins to run

In [17]:
TOTAL_NUM_PATIENTS = 0.1e6 #changed for debugging  

def get_visit_embedding(input_file_path: str): # input_file_path = PATIENT_DATA_FILE
    start_time = time.time()
    visit_encodings = []
    patient_visit = []
    with open(input_file_path, newline='') as csvfile:
        data = csv.reader(csvfile, delimiter=',')
        num_patients = 0
        first_row = True
        curr_patient_id = None
        
        for row in data:
            
            if first_row:
                first_row = False
                continue

            new_patient_id = row[0]
            
            # Used to limit num patients, reducing model space
            if curr_patient_id != new_patient_id:
                
                # If we have enough patients, break
                if num_patients >= TOTAL_NUM_PATIENTS:
                    break
                else:
                    curr_patient_id = new_patient_id
                    num_patients += 1
                    if num_patients % 50000 == 0:
                        print("Reading: ", num_patients, " patients")

            # Convert list of ICD-10 codes to 540-vector one-hot encoding
            code_embedding = rowToEncoding(row).astype(np.int16)
            visit_encodings.append(code_embedding)
            patient_visit.append([int(row[0]), int(row[1])]) # Store patient id and visit id
    
    # Reduce entries to be binary (0 or 1)
    code_dset = np.array(visit_encodings)
    min_arr = np.ones((len(code_dset), len(code_dset[0])))
    assert(min_arr.shape == code_dset.shape)
    code_dset = np.minimum(code_dset, min_arr)

    # Combine patient id, visit id and one-hot encoding
    visit_arr = np.array(patient_visit)
    result = np.hstack((visit_arr, code_dset)) # patient id, visit id, one-hot encoding vector
    return result

code_dset = get_visit_embedding(PATIENT_DATA_FILE)
num_visits = len(code_dset)
code_dset.shape

Invalid secondary code:  129
Reading:  50000  patients
Invalid secondary code:  1510
Reading:  100000  patients


(628402, 542)

In [18]:
# Read in patient_data - takes about a minute
patient_read_start = time.time()
patient_data = pd.read_csv(PATIENT_DATA_FILE, dtype=str, usecols=['ID', 'Sex','Race','Age', 'Date'], nrows = num_visits)
print("Reading in patient_data took {}".format(time.time() - patient_read_start))

# One-hot encode visit features
one_hot_start = time.time()
one_hot_features = create_one_hot(patient_data)
print("Creating one-hot encodings took {}".format(time.time() - one_hot_start))
print(one_hot_features.shape)

Reading in patient_data took 1.1798033714294434
Creating one-hot encodings took 0.3289210796356201
(628402, 11)


## Step 4: Concatenate Feature vector with ICD-10 code encoding

In [19]:
x_dset = np.hstack((code_dset, one_hot_features[:num_visits,:]))

In [20]:
x_dset.shape

(628402, 553)

## Step 5A: Get labels (slow method sorted by pt ID)

To create labels, we convert the 'Date' column into pandas datetime. We then group by patient 'ID' and determine the difference between the next row to the previous. This means if a row value in y_diff is 30 then the next visit is 30 days from the previous. We then set the label to 0 if visit is > 30 days, 1 if it is <= 30 days, or 2 if there is no next visit

In [21]:
def get_y_labels(patient_data):

    patient_data['Date'] = pd.to_datetime(patient_data['Date'])  #convert date column to date-time type
    y_diff = abs(patient_data.groupby(['ID'])['Date'].diff(periods=-1))
    y_diff = y_diff.dt.days
    y_label = []
    for i in y_diff:
        if pd.isnull(i):
            y_label.append(2)
        else:
            if i <= 30:
                y_label.append(1)
            else:
                y_label.append(0)
    return y_label, len(y_label)

In [22]:
# Get y_dset
label_start = time.time()
y_dset, num_labels = get_y_labels(patient_data)
print("Creating labels took {}".format(time.time() - label_start))

Creating labels took 14.187441110610962


In [23]:
# Ensures the number of labels corresponds to the number of patient visits
assert(num_labels == num_visits)

In [24]:
new_dset = np.hstack([x_dset, np.array(y_dset).reshape(len(y_dset),1)])
print(new_dset.shape)

(628402, 554)


In [25]:
del x_dset, y_dset

In [26]:
new_dset = new_dset[new_dset[:,-1]!=2]

In [27]:
print(new_dset.shape)

(528402, 554)


## Step 5B: Get labels (fast method not sorted by pt ID)

To create labels, we convert the 'Date' column into pandas datetime. We then check the difference between rows without sorting. This means if a row is 30 then the next visit is 30 days from the previous. We then set the label to 0 if visit is > 30 days, 1 if it is less than 30 days. The nth visit per patient will be incorrect as it will be using the 1st visit from the next patient to determine the time difference. If we are going to use this we will have to do some processing when we split data by patient and assert that the label for the last visit is 'Na' or '2' or some other marker to show that there is no next visit. 

In [109]:
# def get_y_labels(patient_data):

#     patient_data['Date'] = pd.to_datetime(patient_data['Date'])  #convert date column to date-time type
#     y_diff = abs(patient_data['Date'].diff(periods=-1))
#     y_diff = y_diff.dt.days
#     y_label = []
#     for i in y_diff:
#         if i <= 30:
#             y_label.append(1)
#         else:
#             y_label.append(0)
#     return y_label, len(y_label)

In [110]:
# # Get y_dset
# label_start = time.time()
# y_dset, num_labels = get_y_labels(patient_data)
# print("Creating labels took {}".format(time.time() - label_start))

Creating labels took 0.2488718032836914


In [111]:
# # Ensures the number of labels corresponds to the number of patient visits
# assert(num_labels == num_visits)

## Step 6: Save embeddings and labels

In [32]:
# filename = 'embeddings_{}.pickle'.format(LANG_MODEL)

# start_time = time.time()
# pickle_out = open(filename, 'wb')
# pickle.dump((x_dset, y_dset), pickle_out, protocol=4)
# pickle_out.close()
# print("Saving embeddings took: {}".format(time.time() - start_time))

Saving embeddings took: 119.45295143127441


In [6]:
filename = 'embeddings_{}.pickle'.format(LANG_MODEL)
start_time = time.time()
pickle_in = open(filename, 'rb')
embedding_dset, y_dset = pickle.load(pickle_in)
pickle_in.close()
print("Saving embeddings took: {}".format(time.time() - start_time))

Saving embeddings took: 110.1644721031189


## Step 7: Store patient_id, visit_id, embedding_vec, label in a pandas dataframe

In [30]:
collated_data = new_dset #np.hstack((x_dset, y_dset.reshape(y_dset.shape[0],1))) # -- last col is label column, first 2 are patient an visit ids
cols =  ['patient_id', 'visit_id'] + ['embed_vec'+ str(i) for i in range(new_dset.shape[1]-3)] + ['label']
df = pd.DataFrame(data = collated_data, columns = cols)

In [32]:
df.shape

(528402, 554)

In [34]:
# Clear variables
collated_data = None
new_dset = None
x_dset = None
y_dset = None

Transform the data types 

In [36]:
df['patient_id'] = df['patient_id'].astype('int64')
df['visit_id'] = df['visit_id'].astype('int64')
for col in cols[2:-1]:
    df[col] = df[col].astype(str).astype(np.float32)

df['label'] = df['label'].astype(np.int16)

In [37]:
df.dtypes

patient_id        int64
visit_id          int64
embed_vec0      float32
embed_vec1      float32
embed_vec2      float32
                 ...   
embed_vec547    float32
embed_vec548    float32
embed_vec549    float32
embed_vec550    float32
label             int16
Length: 554, dtype: object

Comment out the pickle dump/load sections below based on need

In [40]:
filename = 'patient_dataframe_03_24_2021.pkl'
pickle_out = open(filename, 'wb')
pickle.dump(df, pickle_out, protocol=4)
pickle_out.close()

In [38]:
# filename = 'patient_dataframe.pkl'
# pickle_in = open(filename, 'rb')
# df = pickle.load(pickle_in)
# pickle_in.close()

FileNotFoundError: [Errno 2] No such file or directory: 'patient_dataframe.pkl'

In [42]:
df.head()

,patient_id,visit_id,embed_vec0,embed_vec1,embed_vec2,embed_vec3,embed_vec4,embed_vec5,embed_vec6,embed_vec7,...,embed_vec542,embed_vec543,embed_vec544,embed_vec545,embed_vec546,embed_vec547,embed_vec548,embed_vec549,embed_vec550,label
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
4,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1


## Step 8: Create the binary classification model

In [45]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

properties = list(df.columns.values)
X = df[properties[2:-1]]
y = df['label']
print(X.shape, len(y))

(528402, 551) 528402


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X.shape[1],)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

## Step 9: Visualiza the data with tSNE/PCA

In [52]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

N = 10000
# For reproducability of the results
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])
feat_cols = list(np.arange(0,df.shape[1]-1)) # replace 0 with 11 for removing demographics
print(feat_cols)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [53]:
pca = PCA(n_components=3)

pca_result = pca.fit_transform(df.iloc[:,feat_cols].values)

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

MemoryError: Unable to allocate 2.18 GiB for an array with shape (553, 528402) and data type float64

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="label",
    palette=sns.color_palette("hls", NUM_CLASSES),
    data=df.loc[rndperm,:],
    legend="full",
    alpha=0.3
)
plt.show()
# plt.savefig('PCA.png')